In [94]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, concatenate, Dense

In [93]:
def InceptionModule(x):
    tower_1 = Conv2D(64, (1, x.shape[2].value), padding='valid', activation='relu')(x)
    tower_1 = Conv2D(64, (3, tower_1.shape[2].value), padding='valid', activation='relu')(tower_1)

    tower_2 = Conv2D(64, (1, x.shape[2].value), padding='valid', activation='relu')(x)
    tower_2 = Conv2D(64, (5, tower_2.shape[2].value), padding='valid', activation='relu')(tower_2)

    tower_3 = MaxPooling2D((3, 1), strides=(1, 1), padding='valid')(x)
    tower_3 = Conv2D(64, (1, tower_3.shape[2].value), padding='valid', activation='relu')(tower_3)
    
    tower_4 = Conv2D(64, (1, x.shape[2].value), padding='valid', activation='relu')(x)

    print(tower_1.shape)
    print(tower_2.shape)
    print(tower_3.shape)
    print(tower_4.shape)
    
    return concatenate([tower_1, tower_2, tower_3, tower_4], axis=1)

In [91]:
windowsize = 100
feature_len = 119

inputs = Input((windowsize, feature_len, 1, ))
x = InceptionModule(inputs)
x = Flatten()(x)
x = Dense(200)(x)
x = Dense(2)(x)
outputs = x

model = Model(input=inputs, output=outputs)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

(?, 98, 1, 64)
(?, 96, 1, 64)
(?, 98, 1, 64)
(?, 100, 1, 64)


c:\users\kevin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()
